In [10]:
import os
print(os.getcwd())  #I'm just gettig how to access the file here

/home/jovyan/work/data-driven-decisions/data-driven-decisions/module_04


In [21]:
##LOAD MATERIALS

import pandas as pd


import pandas as pd

def load_material_properties(filename='/home/jovyan/work/data-driven-decisions/data-driven-decisions/module_04/materials.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        print(f"Error: The file {filename} does not exist.")
    except pd.errors.EmptyDataError:
        print(f"Error: The file {filename} is empty or not formatted correctly.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def get_material_properties(materials_df, material, method):
    row = materials_df[(materials_df['material'] == material) & (materials_df['method'] == method)]
    if not row.empty:
        properties = {
            'youngs_modulus_MPa': row['youngs_modulus_MPa'].values[0],
            'yield_strength_MPa': row['yield_strength_MPa'].values[0],
            'porosity_range': [float(x) for x in row['porosity_range'].values[0].split(',')]
        }
        return properties
    else:
        return None

# Example usage:
materials_df = load_material_properties()
print(materials_df)
material_properties = get_material_properties(materials_df, 'Steel', 'Casting')
print(material_properties)


    material             method  youngs_modulus_MPa  yield_strength_MPa  \
0      Steel            Casting              210000                 550   
1   Aluminum        3D Printing               70000                 250   
2   Titanium          Machining              116000                 880   
3    Polymer  Injection Molding                2500                  50   
4  Composite        3D Printing               70000                 600   

  porosity_range  
0      0.02,0.05  
1      0.05,0.15  
2      0.01,0.03  
3      0.02,0.04  
4       0.03,0.1  
{'youngs_modulus_MPa': 210000, 'yield_strength_MPa': 550, 'porosity_range': [0.02, 0.05]}


In [22]:
##SIMULATION

import numpy as np

def run_monte_carlo_simulation(material_properties,num_simulations):
    """
    Simulate the effects of porosity on a material property.

    Args:
    material (dict): A dictionary containing the material properties and porosity range.
    num_simulations (int): The number of simulations to run.

    Returns:
    np.array: An array of simulated values.
    """
    min_porosity, max_porosity = material_properties['porosity_range']
    # Generate random porosity values within the range
    random_porosities = np.random.uniform(min_porosity, max_porosity, num_simulations)

    # Placeholder for a function that calculates some property affected by porosity
    simulated_values = random_porosities * 0.5  # Example calculation
    return simulated_values



def calculate_porosity_factor(pore_sizes):
    """
    Calculate the porosity factor based on the 75th percentile of pore sizes.

    Args:
    pore_sizes (list or np.array): List of pore diameters.

    Returns:
    float: A porosity factor derived from the pore sizes.
    """
    # Calculate the 75th percentile pore diameter
    p75_diameter = np.percentile(pore_sizes, 75)

    # Example calculation to derive porosity factor (simplified model)
    # Assume porosity factor scales linearly with the diameter of pores
    # This will need to be adjusted based on actual volumetric analysis or experimental data
    porosity_factor = p75_diameter / max(pore_sizes)
    return porosity_factor


def calculate_reduced_life(porosity_factor, expected_life):
    """
    Calculate reduced fatigue life using Goodman, Gerber, and Paris equations.

    Args:
    porosity_factor (float): The porosity factor (0 to 1 scale, where 1 is highly porous).
    expected_life (float): The baseline expected fatigue life without porosity.

    Returns:
    dict: A dictionary containing reduced life estimations according to different models.
    """

    sigma_ut = 400  # Ultimate tensile strength in MPa
    sigma_e = 200  # Endurance limit in MPa
    C = 1e-12  # Paris' law constant C
    m = 3.1  # Paris' law constant m


    # Adjust properties based on porosity
    adjusted_sigma_e = sigma_e * (1 - porosity_factor)
    adjusted_sigma_ut = sigma_ut * (1 - porosity_factor)

    # Goodman's reduction
    goodman_life = expected_life * (1 - porosity_factor)

    # Gerber's reduction
    gerber_life = expected_life * (1 - porosity_factor**2)

    # Paris' Law (simplistic approach for demonstration)
    paris_factor = C * ((adjusted_sigma_e * np.sqrt(1 - porosity_factor))**m)
    paris_life = expected_life / paris_factor

    return {
        'goodman': goodman_life,
        'gerber': gerber_life,
        'paris': paris_life
    }





In [19]:
##USE ABOVE FUNCTIONS/CODE BLOCKS

def main():


    materials_df = load_material_properties()
    material_properties = get_material_properties(materials_df, 'Polymer', 'Injection Molding')

    pore_size_results = run_monte_carlo_simulation(material_properties, 1000)
    print("Simulated Results:", pore_size_results)



    porosity_factor = calculate_porosity_factor(pore_size_results)  #  pore sizes in mm
    expected_life = 1e6  # 1 million cycles without porosity
    reduced_life = calculate_reduced_life(porosity_factor, expected_life)
    print("Porosity Factor:", porosity_factor)
    print("Reduced Fatigue Life (Cycles):", reduced_life)


if __name__ == "__main__":
    main()


Simulated Results: [0.01964091 0.01706578 0.01612837 0.01281927 0.01063437 0.01984276
 0.0115428  0.01174731 0.01123128 0.01717697 0.01821389 0.01682001
 0.01714684 0.0173443  0.01912773 0.01939165 0.01996816 0.01797425
 0.01643799 0.01160121 0.01984416 0.01868227 0.01572879 0.01457774
 0.01690816 0.01191247 0.01685808 0.01654649 0.01849106 0.01426422
 0.01567096 0.01792596 0.01655604 0.01221317 0.01443853 0.01633688
 0.01519123 0.01212272 0.01947721 0.01674754 0.01683569 0.0196213
 0.01532244 0.01720797 0.01558642 0.01201212 0.01048146 0.01568465
 0.01186273 0.01888155 0.01101283 0.01145185 0.01852715 0.01969275
 0.01724958 0.01313482 0.01667437 0.01973149 0.01869172 0.01755025
 0.01368442 0.01146309 0.01292171 0.01638675 0.01004527 0.01253576
 0.01400506 0.01081284 0.01123952 0.01396674 0.01644703 0.01717543
 0.01234467 0.01499126 0.01771888 0.01630401 0.01769549 0.01382943
 0.01717683 0.01136272 0.01593762 0.01095531 0.01593993 0.0139738
 0.01103548 0.01206792 0.01321657 0.01502948 